Para el inicio de este notebook en Databricks primero debemos leer nuestros archivos de la ruta en donde los almacenamos, en mi caso quedaron en Volumes/workspace/default/parcial

In [ ]:
import json
import os
from pyspark.sql import Row
from pyspark.sql import functions as F

# Ruta base del Volume donde subiste los archivos
catalog = "workspace"
schema  = "default"
volume  = "parcial"
BASE = f"/Volumes/{catalog}/{schema}/{volume}"
CSVS = f"{BASE}/csv_out"

arr = os.listdir(BASE)
print(arr)


Convertimos el JSON que cuesta leer en un formato de tipo CSV.

In [ ]:
import json, pandas as pd

IN  = f"/{BASE}/temporadas.json"
OUT = f"/{BASE}/csv_out"            

with open(IN, encoding="utf-8") as f:
    data = json.load(f)

In [ ]:
# ---------- NIVEL EQUIPO/PARTIDO ----------
teams_list = []
HomeTeam = True
for X in data:
    for Y in data[str(X)]:
        team_details = (data[str(X)][str(Y)].get('team_details', {}) or {}).copy()
        aggregate_stats = data[str(X)][str(Y)].get('aggregate_stats', {}) or {}

        team_details.setdefault("team_id", Y)
        team_details['match_id'] = X
        team_details['team'] = "L" if HomeTeam else "V"
        HomeTeam = not HomeTeam

        # merge de agregados
        for k, v in aggregate_stats.items():
            team_details[k] = v

        teams_list.append(team_details)

df_teams = pd.DataFrame(teams_list)

cols_first = ['match_id', 'team_id', 'team_name', 'team']
cols_first = [c for c in cols_first if c in df_teams.columns]
df_teams = df_teams[cols_first + [c for c in df_teams.columns if c not in cols_first]]


In [ ]:
# ---------- NIVEL JUGADOR ----------
players_list = []
for X in data:
    for Y in data[str(X)]:
        players = data[str(X)][str(Y)].get('Player_stats', {}) or {}
        for Z in players:
            d = {}
            pdet = (players[str(Z)].get('player_details', {}) or {})
            pms  = (players[str(Z)].get('Match_stats', {}) or {})

            d.update(pdet)
            d.update(pms)
            d['match'] = X
            d['team'] = Y
            d['player_name'] = pdet.get('player_name') or str(Z)

            players_list.append(d)

df_players = pd.DataFrame(players_list)

cols_first = ['match', 'team', 'player_name']
cols_first = [c for c in cols_first if c in df_players.columns]
df_players = df_players[cols_first + [c for c in df_players.columns if c not in cols_first]]


In [ ]:
# ---------- GUARDAR ----------
import os
os.makedirs(OUT, exist_ok=True)
df_teams.to_csv(f"{OUT}/TeamsJSON.csv", index=False, encoding="utf-8")
df_players.to_csv(f"{OUT}/JugadoresJSON.csv", index=False, encoding="utf-8")
print("Listo:", f"{OUT}/TeamsJSON.csv", f"{OUT}/JugadoresJSON.csv")

In [ ]:
jug = (spark.read.option("header","true").option("inferSchema","true").csv(f"{BASE}/jugadores.csv"))
res_futbol = (spark.read.option("header","true").option("inferSchema","true").csv(f"{BASE}/resultados_futbol.csv"))  
equ_json = (spark.read.option("header","true").option("inferSchema","true").csv(f"{BASE}/csv_out/TeamsJSON.csv"))
jug_json = (spark.read.option("header","true").option("inferSchema","true").csv(f"{BASE}/csv_out/JugadoresJSON.csv"))

display(jug.limit(5))
display(res_futbol.limit(5))
display(equ_json.limit(5))
display(jug_json.limit(5))

### 

### Glosario jugadores.csv

match_id (int): identificador único del partido.

team_id (int): identificador del equipo según el proveedor.

team_name (str): nombre del equipo.

team (str): local/visitante (suele venir como L = local, V = visitante).

team_rating (double): calificación agregada del equipo basada en acciones del partido (algoritmo propietario; en proveedores tipo SofaScore las ratings se calculan automáticamente a partir de eventos). 

date (date): fecha del partido.

won_corners (int): saques de esquina ganados por el equipo (equivale a “HC/AC” en algunos ficheros). 
Football Data

att_sv_low_centre (int): intentos a puerta (on target) salvados por el portero dirigidos a la zona baja-central de la portería (nomenclatura habitual en datos estilo Opta; la “baja-central” es una de las zonas de colocación de tiro usadas por estos proveedores). 

won_contest (int): duelos ganados (incluye duelos aéreos/terrestres; depende del proveedor). 

total_tackle (int): entradas realizadas (tackles, ganadas o totales según la implementación). 

aerial_lost (int): duelos aéreos perdidos. 

possession_percentage (double): % de posesión del balón del equipo (método varía por proveedor: por tiempo efectivo o por secuencias/pases). 

accurate_pass (int): pases acertados/completados (paso que llega a un compañero).


### Glosario resultados_futbol.csv

name (str): nombre del jugador.

club (str): club actual.

age (int): edad.

position (str): posición principal (p. ej., DF/MF/FW/GK).

position_cat (int): categoría numérica de posición (feature de dataset; p. ej., 1=GK, 2=DF, 3=MF, 4=FW; confirmar mapeo exacto en tu fuente).

market_value (double): valor de mercado estimado (frecuente en Transfermarkt; es una estimación comunitaria, no una tarifa de traspaso). 

page_views (int): vistas de página (suele ser popularidad, a menudo de Wikipedia si el dataset lo incluye). 

fpl_value (double): “valor” en Fantasy Premier League (comúnmente puntos totales / precio en millones). 

fpl_sel (str): % “Selected by” (porcentaje de mánagers FPL que poseen al jugador). 

fpl_points (int): puntos FPL acumulados según el sistema oficial. 

region / nationality (str): región/país.

new_foreign (int): indicador de jugador extranjero recién llegado (feature del dataset; confirmar lógica exacta).

age_cat (int): bines de edad (feature del dataset).

club_id (int): id del club en el proveedor.

big_club (int): indicador de “club grande” (feature del dataset; p. ej., Big-6, etc.).

new_signing (int): fichaje reciente (feature del dataset).

### Glosario TeamsJSON.csv

match (int): id del partido.

team (int/str): id/código del equipo en ese partido.

player_name (str), player_id (int).

player_position_value (int) / player_position_info (str): posición o rol en ese partido.

player_rating (double): rating del rendimiento del jugador (algoritmo propietario; en sitios como SofaScore/WhoScored se recalcula en tiempo real con eventos). 

good_high_claim (int): “alta captura buena” del portero: alto balón al área atrapado con éxito (claim) — se distingue de “punch” (puñetazo) y de “cross not claimed” (salida fallida).

touches (int): toques de balón (suma de eventos donde el jugador toca el balón). 

saves (int): paradas del portero (tiros a puerta detenidos). 

total_pass (int): pases intentados.

formation_place (int): posición en el 11 según el dibujo/alineación (número de “casilla” en la formación; convención del proveedor). (No hay estándar universal; es específico de la data).

accurate_pass (int): pases acertados/completados. 

aerial_won / aerial_lost (int): duelos aéreos ganados/perdidos. 

fouls (int): faltas cometidas (según criterio Opta/gobernadas por el árbitro). 

total_scoring_att (int): tiros totales (attempts). 

total_tackle (int): entradas realizadas (ver definición arriba). 

won_contest (int): duelos ganados. 

penalty_conceded (int): penaltis concedidos por el jugador. 

blocked_scoring_att (int): tiros bloqueados por defensores. 

man_of_the_match (int): indicador de premio “Jugador del partido” (según proveedor).

total_throws (int): saques de banda ejecutados.

shot_off_target (int): tiros fuera. 

ontarget_scoring_att (int): tiros a puerta (incluye goles y tiros detenidos; si un defensa bloquea en la línea, muchos proveedores lo consideran on target). 

att_miss_left (int): tiros fuera – zona izquierda (clasificación por colocación).

### Glosario JugadoresJSON.csv

Season (str): temporada (p. ej., “2018/2019”).

DateTime (ts): fecha y hora del kickoff.

HomeTeam / AwayTeam (str): equipos local/visitante.

FTHG / FTAG (int): goles al final del partido (local/visitante).

FTR (str): resultado final (H=local, D=empate, A=visitante).

HTHG / HTAG (str/int): goles al descanso (local/visitante).

HTR (str): resultado al descanso (H/D/A).

Referee (str): árbitro.

HS / AS (str/int): tiros del local/visitante.

HST / AST (str/int): tiros a puerta del local/visitante.

HC / AC (str/int): córners local/visitante.

HF / AF (str/int): faltas cometidas local/visitante.

HY / AY (str/int): amarillas local/visitante.

HR / AR (str/int): rojas local/visitante.